# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# | export

import reader
from typing import *
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s)
    return s.text


class PyNewsReader(reader.Reader):
    def __init__(self, dbpath=None, feeds=None):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self.reader = reader.make_reader(
            "db.sqlite", plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        if feeds is None or self.reader.get_feed_counts().total == 0:
            feeds = [
                "https://rss.cbc.ca/lineup/topstories.xml",
                "https://rss.cbc.ca/lineup/world.xml",
                "https://rss.cbc.ca/lineup/canada.xml",
                "https://rss.cbc.ca/lineup/business.xml",
                "https://rss.cbc.ca/lineup/technology.xml",
                "https://www.cbc.ca/cmlink/rss-canada-ottawa",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
                "https://www.macleans.ca/feed/",
                "https://thetyee.ca/rss2.xml",
                "https://thenarwhal.ca/feed/",
                "https://ricochet.media/en/feed",
            ]

        for feed in feeds:
            logger.info("Feeds not specified, starting you off with some Canadian news")
            self.reader.add_feed(feed, exist_ok=True)

        super().__init__(
            self.reader._storage,
            self.reader._search,
            self.reader._parser,
            self.reader._reserved_name_scheme,
        )
        self.enable_search()

    def _print_entries(self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                r.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = Text(
                        "Date: " + e.published.isoformat()[:10], justify="center"
                    )
                else:
                    published_date = Text("Date: Unknown", justify="center")
                if mark_as_read:
                    r.mark_entry_as_read(e)
                print(
                    Panel(
                        published_date
                        + "\n\n"
                        + Text(strip_html(e.summary) + "\n", justify=None),
                        title=f"[link={e.link}]{e.title}[/link]",
                        subtitle=e.feed.title or e.feed_url,
                    )
                )
                print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in r.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def show(self, limit: int = 5, read: bool = None, **kwargs) -> type(None):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        self._print_entries(self.get_entries(**kwargs, read=read, limit=limit*2), limit=limit)

    def search(self, query: str, mark_as_read: bool = True):
        """Search entries and pretty print results"""
        entries = []
        for e in r.search_entries(query):
            if mark_as_read:
                r.mark_entry_as_read(e)
            entries.append(self._search_to_entry(e))
        self._print_entries(entries)

    def update(self):
        """Update feeds and search"""
        self.update_feeds()
        self.update_search()

In [ ]:
r = PyNewsReader()

/home/mark/anaconda3/lib/python3.8/site-packages/reader/core.py:443: UserWarning: Reader objects should be created using make_reader(); the Reader constructor is not stable yet and may change without any notice.
  warnings.warn(


## List feeds

In [ ]:
[i.title or i.url for i in r.get_feeds()]

['https://ricochet.media/en/feed',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans.ca',
 'The Narwhal',
 "The Tyee | BC's home for today's news, analysis and updates",
 'TORONTO STAR',
 'TORONTO STAR',
 'TORONTO STAR']

## Update Feeds

In [ ]:
r.update()

## Get Entries

In [ ]:
for i in r.get_entries(limit=1):
    print(i)

Entry(
    id='1.6684943',
    updated=None,
    title='Multiple injuries reported after bus, car collide northeast of Quebec City',
    link='https://www.cbc.ca/news/canada/montreal/bus-car-collide-quebec-1.6684943?cmp=rss',
    author='CBC News',
    published=datetime.datetime(2022, 12, 14, 0, 32, 56, tzinfo=datetime.timezone.utc),
    summary='<img alt="collision" height="259" 
src="https://i.cbc.ca/1.6684973.1670978692!/fileImage/httpImage/image.JPG_gen/derivatives/16x9_460/collision.JPG" 
title="Sûreté du Québec police and ambulances were called to the scene after a bus and car collided near 
Saint-Fabien, Que." width="460" />                <p>The Orléans Express bus and car collided around 5:15 p.m. on 
route 132, near Saint-Fabien, Que., about 285 kilometres northeast of Quebec City.\xa0There were about 40 
passengers on the bus.</p>',
    content=(),
    enclosures=(),
    read=True,
    read_modified=datetime.datetime(2022, 12, 14, 1, 43, 6, 888791, tzinfo=datetime.timezone.utc),
    important=False,
    important_modified=None,
    added=datetime.datetime(2022, 12, 14, 1, 43, 1, 799593, tzinfo=datetime.timezone.utc),
    added_by='feed',
    last_updated=datetime.datetime(2022, 12, 14, 1, 43, 1, 799593, tzinfo=datetime.timezone.utc),
    original_feed_url='https://rss.cbc.ca/lineup/topstories.xml',
    feed=Feed(
        url='https://rss.cbc.ca/lineup/topstories.xml',
        updated=datetime.datetime(2022, 12, 14, 0, 32, 56, tzinfo=datetime.timezone.utc),
        title='CBC | Top Stories News',
        link='http://www.cbc.ca/news/?cmp=rss',
        author=None,
        subtitle='FOR PERSONAL USE ONLY',
        version='rss20',
        user_title=None,
        added=datetime.datetime(2022, 12, 7, 0, 40, 58, 106810, tzinfo=datetime.timezone.utc),
        last_updated=datetime.datetime(2022, 12, 14, 1, 43, 1, 799593, tzinfo=datetime.timezone.utc),
        last_exception=None,
        updates_enabled=True
    )
)

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None)

╭────────────────── ]8;id=344643;https://www.cbc.ca/news/canada/montreal/bus-car-collide-quebec-1.6684943?cmp=rss\Multiple injuries reported after bus, car collide northeast of Quebec City]8;;\ ───────────────────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│    The Orléans Express bus and car collided around 5:15 p.m. on route 132, near Saint-Fabien, Que., about 285   │
│                 kilometres northeast of Quebec City. There were about 40 passengers on the bus.                 │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭─────── ]8;id=480511;https://www.cbc.ca/news/politics/hockey-canada-public-financial-statements-1.6684855?cmp=rss\Hockey Canada to make financial statements public after backlash over sexual assault settlements]8;;\ ────────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│  Hockey Canada says it will release its audited financial statements following a public outcry over its use of  │
│            controversial reserve funds that could be employed to settle sexual assault allegations.             │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭───────────────────── ]8;id=607363;https://www.cbc.ca/news/canada/british-columbia/b-c-men-in-violent-protests-in-peru-1.6684390?cmp=rss\Victoria, B.C., men recount being caught in violent protests in Peru]8;;\ ──────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  Peru has been sinking deeper into political crisis amid violent protests following the impeachment and arrest  │
│                                   of leftist leader President Pedro Castillo.                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

╭─ ]8;id=602424;https://www.thestar.com/politics/federal/2022/12/13/pierre-poilievre-needs-to-win-ridings-like-mississauga-lakeshore-insiders-reveal-why-he-didnt-try-harder-this-time.html\Pierre Poilievre needs to win ridings like Mississauga-Lakeshore. Insiders reveal why he didn’t try harder thi]8;;\─╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  Voters in Mississauga-Lakeshore have returned a Liberal to Parliament in a byelection that opposition parties  │
│                                              played down Tuesday.                                               │
│                                                                                                                 │
╰───────────────────────────────────────────────── TORONTO STAR ──────────────────────────────────────────────────╯

╭───────────────── ]8;id=550853;https://www.cbc.ca/news/canada/north/atlin-bc-postmaster-resigns-1.6684360?cmp=rss\The only postmaster in Atlin, B.C., has a message for Canada Post — she quits]8;;\ ─────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│    After a contract dispute, Tina St Cyr says she'll finish as postmaster on Dec. 23. It's unclear what will    │
│                             happen with mail delivery in the community after that.                              │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

## Search Entries

In [ ]:
r.search("fusion")

╭──────────────────────── ]8;id=208537;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\Here's why a breakthrough in nuclear fusion is such a big deal]8;;\ ─────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

╭─────────────────────────── ]8;id=251681;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\Fusion energy 'breakthrough' revealed by U.S. scientists]8;;\ ────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│      U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the      │
│                 decades-long quest to harness fusion, the energy that powers the sun and stars.                 │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────── ]8;id=408568;https://www.thestar.com/news/canada/2022/12/13/scientists-reveal-holy-grail-breakthrough-to-create-limitless-clean-energy.html\Scientists reveal ‘holy grail’ breakthrough to create limitless clean energy]8;;\ ──────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  Scientists have taken the first step towards cheap, clean and near unlimited power by harnessing the power of  │
│                                the sun, announced the U.S. Department of Energy.                                │
│                                                                                                                 │
╰───────────────────────────────────────────────── TORONTO STAR ──────────────────────────────────────────────────╯

In [ ]:
#| hide

## To Do:
## * Change title of feed
## * Command line interface
## * User interface
## * Share / favourite entries
## * Tags

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()